<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC2_FD001_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
%pip install scikeras;
%pip install -U tensorflow-addons;
%pip install scikit-optimize


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import datetime

H:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
H:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import GroupKFold

from sklearn.base import BaseEstimator,RegressorMixin

In [4]:
from skopt import BayesSearchCV

from skopt.space.space import Categorical, Integer, Real

In [5]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Masking

from tensorflow.keras.metrics import RootMeanSquaredError as RMSE

from keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import tensorflow_addons as tfa
from tensorflow_addons.metrics import RSquare as R2

from scikeras.wrappers import KerasRegressor

In [6]:
sns.set_palette('colorblind')

In [7]:
# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [8]:
tf.config.experimental.enable_op_determinism()

In [9]:
# Remove some tf warnings
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

# Data Preparation

In [10]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    IN_COLAB = True
    folder="/content/"

    # Dataset Download 
    os.system('git clone https://github.com/arthursl12/dataset_2')
    os.system('mv /content/dataset_2/CMaps /content/CMaps')
    os.system('mv /content/dataset_2/data_processing /content/data_processing')
    os.system('rm -rf dataset_2')
else:
    print('Not running on CoLab')
    IN_COLAB = False
    folder="CMaps/"
    %cd dataset_2/

Not running on CoLab
C:\Users\Arthur Lima\POC\dataset_2


In [11]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [12]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [13]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(1, folder='CMaps/')
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640


## Preprocessing

### Validation Set

In [14]:
from sklearn.model_selection import GroupShuffleSplit

def train_val_split(train):
    gss = GroupShuffleSplit(n_splits=1, train_size=0.80, random_state=42)  
    for idx_train, idx_val in gss.split(train,groups=train["unit_number"]):
        # print('train_split_engines', train.iloc[idx_train]['unit_number'].unique(), '\n')
        # print('validate_split_engines', train.iloc[idx_val]['unit_number'].unique(), '\n')

        df_train = train.iloc[idx_train].copy()
        df_val = train.iloc[idx_val].copy()

    return df_train, df_val

In [15]:
df_train, df_val = train_val_split(train)

In [16]:
X_t_train, y_t_train = proc.X_y_train_divide(df_train)
X_t_val, y_t_val = proc.X_y_train_divide(df_val)

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [17]:
test.shape, y_test.shape

((13096, 26), (100, 1))

In [18]:
test_last = proc.transform_test(test)
test_last.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.58,1581.22,1398.91,14.62,21.61,554.42,2388.08,9056.40,1.3,...,521.79,2388.06,8130.11,8.4024,0.03,393,2388,100.0,38.81,23.3552
1,518.67,642.55,1586.59,1410.83,14.62,21.61,553.52,2388.10,9044.77,1.3,...,521.74,2388.09,8126.90,8.4505,0.03,391,2388,100.0,38.81,23.2618
2,518.67,642.88,1589.75,1418.89,14.62,21.61,552.59,2388.16,9049.26,1.3,...,520.83,2388.14,8131.46,8.4119,0.03,395,2388,100.0,38.93,23.2740
3,518.67,642.78,1594.53,1406.88,14.62,21.61,552.64,2388.13,9051.30,1.3,...,521.88,2388.11,8133.64,8.4634,0.03,395,2388,100.0,38.58,23.2581
4,518.67,642.27,1589.94,1419.36,14.62,21.61,553.29,2388.10,9053.99,1.3,...,521.00,2388.15,8125.74,8.4362,0.03,394,2388,100.0,38.75,23.4117


In [19]:
X_test = test_last

### Remaining Useful Life (RUL)

In [20]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,191
1,1,2,190
2,1,3,189
3,1,4,188
4,1,5,187


## Attributes and target separation

In [21]:
X_train, y_train = proc.X_y_train_divide(train)

In [22]:
y_train.head()

,RUL
0,191
1,190
2,189
3,188
4,187


In [23]:
X_train.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


## Training and Evaluation functions

In [24]:
eval = Evaluation()

In [25]:
search = HyperparameterSearch()

# LSTM Construction

## Callbacks

In [26]:
# Early Stopping Callback
es = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                      patience=5, restore_best_weights=True)

In [27]:
# Printing Callback
def printLog(epoch, logs):
    print(
        f"E {epoch+1}\t: loss={logs['loss']:.3f}, "+
        f"rmse={logs['root_mean_squared_error']:.3f}, "+
        f"r2={logs['r_square']:.3f}; "+
        f"v_loss={logs['val_loss']:.3f}, "+
        f"v_rmse={logs['val_root_mean_squared_error']:.3f}, "+
        f"v_r2={logs['val_r_square']:.3f}; "
    )

printerCallback = LambdaCallback(on_epoch_end=printLog)

## Auxiliary HyperParameters

In [28]:
layer_sizes=[16,32,64,128,256,512]

In [29]:
# Exponential Decay Schedules
ED1 = ExponentialDecay(initial_learning_rate=1e-2, name="ED1",
                       decay_steps=100000, decay_rate=0.96)
ED2 = ExponentialDecay(initial_learning_rate=1e-2, name="ED2",
                       decay_steps=100000, decay_rate=0.8)
ED3 = ExponentialDecay(initial_learning_rate=1e-1, name="ED3",
                       decay_steps=100000, decay_rate=0.96)

In [30]:
# Generation of list combinations
from itertools import chain, permutations

def all_permutations(lst, size):
    result = list(chain.from_iterable([permutations(lst, x) for x in range(len(lst)+1)]))
    out = []
    for r in result:
        if (len(r) == size):
            out.append(list(r))
    return out

## Constants

In [31]:
# X_train must include indices
train3 = train.copy()
X_train_ = train3.drop(columns=["RUL"])

In [32]:
INPUT_SHAPE = 0

## Wrapper

In [33]:
from sklearn.metrics import r2_score

class LSTMWrapperRegressor(BaseEstimator,RegressorMixin):
    def __init__(self, basemodel=None, clip_y=-1, seq_length=40,
                 include_settings=False, poly_degree=1,
                 scaler=StandardScaler()):
        # Base parameters
        self.basemodel = basemodel
        self.clip_y = clip_y
        self.seq_length = seq_length
        self.poly_degree = poly_degree
        self.include_settings = include_settings

        # Column indexers
        self.feature_cols = sensors_cols
        if(include_settings):
            # self.seq_cols = settings_cols + self.cols
            self.feature_cols = settings_cols + self.feature_cols
        self.seq_cols = ["time"] + self.feature_cols
        self.base_feature_cols = self.feature_cols

        # Scaler and PolyFeatures transformers
        self.scaler = scaler
        self.polyft = PolynomialFeatures(degree=self.poly_degree, 
                                         include_bias=False)



    def fit(self, X=None, y=None):
        # Merge features and target again
        data = X.copy()
        data["RUL"] = y

        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.fit_transform(data[self.base_feature_cols])
        transf = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out(),
                              index=data.index)
        self.feature_cols = list(self.polyft.get_feature_names_out())
        self.seq_cols = ["time"] + self.feature_cols
        data[self.feature_cols] = transf

        # Scale the data
        data[self.feature_cols] = \
                            self.scaler.fit_transform(data[self.feature_cols])

        # Transform into time series
        X_train = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        # print(data.shape,X_train.shape, X.shape)

        # Clip and transform labels
        data2 = data.copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_train = self.gen_y_wrapper(data2,self.seq_length,["RUL"])

        # Update input shape for future use
        global INPUT_SHAPE
        # print(INPUT_SHAPE, X_train.shape)
        INPUT_SHAPE = (X_train.shape[1],X_train.shape[2])

        # Fit model
        # print(X_train.shape, y_train.shape)
        self.basemodel.fit(X_train,y_train)
        return self
        
    def predict(self, X=None):
        # Perform transformation, if not done
        if (len(X.shape) < 3):
            data = X.copy()

            # Apply polynomial features
            transf = self.polyft.transform(data[self.base_feature_cols])
            transf = pd.DataFrame(transf, 
                                  columns=self.polyft.get_feature_names_out(),
                                  index=data.index)
            # data = pd.concat([data,transf], axis=1)
            data[self.feature_cols] = transf

            # Scale the data
            data[self.feature_cols] = \
                            self.scaler.transform(data[self.feature_cols])
            
            # Transform into time series
            X_train = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        else:
            X_train = X
        return self.basemodel.predict(X_train)

    def score(self, X, y, sample_weight=None):
        # Merge features and target again
        data = X.copy()
        data["RUL"] = y

        # Apply polynomial features
        transf = self.polyft.transform(data[self.base_feature_cols])
        transf = pd.DataFrame(transf, 
                              columns=self.polyft.get_feature_names_out(),
                              index=data.index)
        # data = pd.concat([data,transf], axis=1)
        data[self.feature_cols] = transf

        # Scale the data (with train data parameters)
        data[self.feature_cols] = \
                        self.scaler.transform(data[self.feature_cols])
        
        # Transform into time series
        X_test = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        # print(data.shape,X_test.shape, X.shape)

        # Clip and transform labels
        data2 = data.copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_test = self.gen_y_wrapper(data2,self.seq_length,["RUL"])

        # Predict on test data
        y_pred = self.predict(X_test)
        return r2_score(y_test, y_pred, sample_weight=sample_weight)

    def gen_X_data(self, df, sequence_length, columns, mask_value=-99.):
        if df.shape[0] < sequence_length:
            # print("\t Not enough sequence:",df.shape[0]," < ",sequence_length)
            data = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
            idx = data.shape[0] - df.shape[0]
            data[idx:,:] = df[columns].values  # fill with available data
        else:
            data = df[columns].values
            
        # # specifically yield the last possible sequence
        # stop = num_elements = data_matrix.shape[0]
        # start = stop - sequence_length
        # for i in list(range(1)):
        #     yield data_matrix[start:stop, :]



        # data = df[columns].values
        num_elements = data.shape[0]

        # -1 and +1 because of Python indexing
        for start, stop in zip(range(0, num_elements-(sequence_length-1)), 
                               range(sequence_length, num_elements+1)):
            yield data[start:stop, :]

    def gen_X_wrapper(self, df, sequence_length, columns, 
                      unit_nrs=np.array([]), idx_col="unit_number"):
        # print(">> Wrapper called. df shape:",df.shape)
        if unit_nrs.size <= 0:
            unit_nrs = df[idx_col].unique()
            
        data_gen = (list(self.gen_X_data(df[df[idx_col]==unit_nr], 
                                         sequence_length, columns))
                for unit_nr in unit_nrs)
        # print("\tdatagen len:",len(data_gen))
        data_array = np.concatenate(list(data_gen)).astype(np.float32)
        # print("\tdata_array.shape:",data_array.shape)
        return data_array

    def gen_y(self, df, sequence_length, label):
        data_matrix = df[label].values
        num_elements = data_matrix.shape[0]

        # -1 because I want to predict the rul of that last row in the sequence, 
        # not the next row
        return data_matrix[sequence_length-1:num_elements, :]  

    def gen_y_wrapper(self, df, sequence_length, label, 
                      unit_nrs=np.array([]), idx_col="unit_number"):
        # print(">> Y Wrapper called. df shape:",df.shape)
        if unit_nrs.size <= 0:
            unit_nrs = df[idx_col].unique()
            
        label_gen = [self.gen_y(df[df[idx_col]==unit_nr], 
                                sequence_length, label) 
                    for unit_nr in unit_nrs]
        # print("\tlabelgen len:",len(label_gen))
        label_array = np.concatenate(label_gen).astype(np.float32)
        # print("\tlabel_array.shape:",label_array.shape)
        return label_array

In [34]:
from sklearn.metrics import r2_score

class LSTMWrapperRegressor2(BaseEstimator,RegressorMixin):
    def __init__(self, basemodel=None, clip_y=-1, seq_length=40,
                 include_settings=False, poly_degree=1,
                 scaler=StandardScaler()):
        # Base parameters
        self.basemodel = basemodel
        self.clip_y = clip_y
        self.seq_length = seq_length
        self.poly_degree = poly_degree
        self.include_settings = include_settings

        # Column indexers
        self.feature_cols = sensors_cols
        if(include_settings):
            # self.seq_cols = settings_cols + self.cols
            self.feature_cols = settings_cols + self.feature_cols
        self.seq_cols = ["time"] + self.feature_cols
        self.base_feature_cols = self.feature_cols

        # Scaler and PolyFeatures transformers
        self.scaler = scaler
        self.polyft = PolynomialFeatures(degree=self.poly_degree, 
                                         include_bias=False)



    def fit(self, X=None, y=None):
        # Merge features and target again
        data = X.copy()
        data["RUL"] = y

        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.fit_transform(data[self.base_feature_cols])
        transf = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out(),
                              index=data.index)
        data = data.drop(self.feature_cols, axis=1)
        
        self.feature_cols = list(self.polyft.get_feature_names_out())
        self.seq_cols = ["time"] + self.feature_cols
        print(all(data.index==transf.index))
        data = pd.concat([data,transf], axis=1)
        # data[self.feature_cols] = transf

        # Scale the data
        transf = self.scaler.fit_transform(data[self.feature_cols])
        transf = pd.DataFrame(transf, columns=
                              self.scaler.get_feature_names_out(),
                              index=data.index)
        data = data.drop(self.feature_cols, axis=1)
        data = pd.concat([data,transf], axis=1)

        # Transform into time series
        X_train = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        # print(data.shape,X_train.shape, X.shape)

        # Clip and transform labels
        data2 = data.copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_train = self.gen_y_wrapper(data2,self.seq_length,["RUL"])

        # Update input shape for future use
        global INPUT_SHAPE
        # print(INPUT_SHAPE, X_train.shape)
        INPUT_SHAPE = (X_train.shape[1],X_train.shape[2])

        # Fit model
        print(X_train.shape, y_train.shape)
        self.basemodel.fit(X_train,y_train)
        return self
        
    def predict(self, X=None):
        # Perform transformation, if not done
        if (len(X.shape) < 3):
            data = X.copy()

            # Apply polynomial features
            transf = self.polyft.transform(data[self.base_feature_cols])
            transf = pd.DataFrame(transf, 
                                  columns=self.polyft.get_feature_names_out(),
                                  index=data.index)
            # data = pd.concat([data,transf], axis=1)
            data[self.feature_cols] = transf

            # Scale the data
            data[self.feature_cols] = \
                            self.scaler.transform(data[self.feature_cols])
            
            # Transform into time series
            X_train = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        else:
            X_train = X
        return self.basemodel.predict(X_train)

    def score(self, X, y, sample_weight=None):
        # Merge features and target again
        data = X.copy()
        data["RUL"] = y

        # Apply polynomial features
        transf = self.polyft.transform(data[self.base_feature_cols])
        transf = pd.DataFrame(transf, 
                              columns=self.polyft.get_feature_names_out(),
                              index=data.index)
        # data = pd.concat([data,transf], axis=1)
        data[self.feature_cols] = transf

        # Scale the data (with train data parameters)
        data[self.feature_cols] = \
                        self.scaler.transform(data[self.feature_cols])
        
        # Transform into time series
        X_test = self.gen_X_wrapper(data,self.seq_length,self.seq_cols)
        # print(data.shape,X_test.shape, X.shape)

        # Clip and transform labels
        data2 = data.copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_test = self.gen_y_wrapper(data2,self.seq_length,["RUL"])

        # Predict on test data
        y_pred = self.predict(X_test)
        return r2_score(y_test, y_pred, sample_weight=sample_weight)

    def gen_X_data(self, df, sequence_length, columns, mask_value=-99.):
        if df.shape[0] < sequence_length:
            # print("\t Not enough sequence:",df.shape[0]," < ",sequence_length)
            data = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
            idx = data.shape[0] - df.shape[0]
            data[idx:,:] = df[columns].values  # fill with available data
        else:
            data = df[columns].values
            
        # # specifically yield the last possible sequence
        # stop = num_elements = data_matrix.shape[0]
        # start = stop - sequence_length
        # for i in list(range(1)):
        #     yield data_matrix[start:stop, :]



        # data = df[columns].values
        num_elements = data.shape[0]

        # -1 and +1 because of Python indexing
        for start, stop in zip(range(0, num_elements-(sequence_length-1)), 
                               range(sequence_length, num_elements+1)):
            yield data[start:stop, :]

    def gen_X_wrapper(self, df, sequence_length, columns, 
                      unit_nrs=np.array([]), idx_col="unit_number"):
        # print(">> Wrapper called. df shape:",df.shape)
        if unit_nrs.size <= 0:
            unit_nrs = df[idx_col].unique()
            
        data_gen = (list(self.gen_X_data(df[df[idx_col]==unit_nr], 
                                         sequence_length, columns))
                for unit_nr in unit_nrs)
        # print("\tdatagen len:",len(data_gen))
        data_array = np.concatenate(list(data_gen)).astype(np.float32)
        # print("\tdata_array.shape:",data_array.shape)
        return data_array

    def gen_y(self, df, sequence_length, label):
        data_matrix = df[label].values
        num_elements = data_matrix.shape[0]

        # -1 because I want to predict the rul of that last row in the sequence, 
        # not the next row
        return data_matrix[sequence_length-1:num_elements, :]  

    def gen_y_wrapper(self, df, sequence_length, label, 
                      unit_nrs=np.array([]), idx_col="unit_number"):
        # print(">> Y Wrapper called. df shape:",df.shape)
        if unit_nrs.size <= 0:
            unit_nrs = df[idx_col].unique()
            
        label_gen = [self.gen_y(df[df[idx_col]==unit_nr], 
                                sequence_length, label) 
                    for unit_nr in unit_nrs]
        # print("\tlabelgen len:",len(label_gen))
        label_array = np.concatenate(label_gen).astype(np.float32)
        # print("\tlabel_array.shape:",label_array.shape)
        return label_array

## Test Data

In [35]:
def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:,:] = df[columns].values  # fill with available data
    else:
        data_matrix = df[columns].values
        
    # specifically yield the last possible sequence
    stop = num_elements = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]

In [36]:
def gen_test_wrapper(X_test_scaled, sequence_length, cols, idx_col="unit_number"): 
    data_gen = (
        list(gen_test_data(X_test_scaled[X_test_scaled[idx_col]==unit_nr], 
                           sequence_length, cols, -99.))
            for unit_nr in X_test_scaled[idx_col].unique())
    data_array = np.concatenate(list(data_gen)).astype(np.float32)
    return data_array

In [37]:
def scale_test(test,model):
    test2 = test.copy()

    # Apply polynomial features
    transf = model.polyft.transform(test2[model.base_feature_cols])
    transf = pd.DataFrame(transf, 
                          columns=model.polyft.get_feature_names_out(),
                          index=test2.index)
    newcols = model.polyft.get_feature_names_out()
    test2[newcols] = transf 

    # Scale the data (with train data parameters)
    test2[model.feature_cols] = \
                    model.scaler.transform(test2[model.feature_cols])
    return test2

## Constructor

In [43]:
def create_model(optim=Adam, learning_rate=1e-3, 
                 layer1=32  , activation1="tanh"    , dropout1=0.1,
                 layer2=None, activation2="tanh"    , dropout2=0.1,
                 layer3=None, activation3="tanh"    , dropout3=0.1,
                 second_dense=True,
                 print_summary=False, loss='mean_squared_error',
                 metrics=[tf.keras.metrics.MeanSquaredError()]):
    model = Sequential()

    # Input-masked layer
    model.add(Masking(mask_value=-99., input_shape=INPUT_SHAPE))
    
    if (layer2 is None and layer3 is None):
        # Single LSTM layer
        model.add(LSTM(layer1, activation=activation1))
        model.add(Dropout(dropout1))
    elif (layer2 is not None and layer3 is None):
        # LSTM-Dense
        model.add(LSTM(layer1, activation=activation1))
        model.add(Dropout(dropout1))
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
    elif (layer2 is not None and layer3 is not None and second_dense==False):
        # LSTM-LSTM-Dense
        model.add(LSTM(layer1, activation=activation1, return_sequences=True))
        model.add(Dropout(dropout1))
        model.add(LSTM(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        model.add(Dense(layer3, activation=activation3))
        model.add(Dropout(dropout3))
    elif (layer2 is not None and layer3 is not None and second_dense==True):
        # LSTM-Dense-Dense
        model.add(LSTM(layer1, activation=activation1))
        model.add(Dropout(dropout1))
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        model.add(Dense(layer3, activation=activation3))
        model.add(Dropout(dropout3))

    # Output Layer
    model.add(Dense(1))

    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), 
                  metrics=metrics)
    
    if(print_summary): model.summary()
    return model

# LSTM 1-layer

## Linear RUL


Score: 0.7965826139501325  
Test: 0.706
```
('basemodel__batch_size', 32),
('basemodel__epochs', 28),
('basemodel__model__activation', 'tanh'),
('basemodel__model__dropout', 0.10771222326909816),
('basemodel__model__layer1', 505),
('basemodel__model__learning_rate', 0.0032806529941975817),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.1),
('scaler', MinMaxScaler()),
('seq_length', 70)
```



In [ ]:
SEQ_LENGTH=70
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=MinMaxScaler(),
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=23,
                           model__activation1='tanh',
                           model__dropout1=0.10771222326909816, 
                           model__layer1=505, 
                           model__learning_rate=0.0032806529941975817,
                           model__optim=RMSprop,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

True
(13731, 70, 22) (13731, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 70, 22)            0         
                                                                 
 lstm (LSTM)                 (None, 505)               1066560   
                                                                 
 dropout (Dropout)           (None, 505)               0         
                                                                 
 dense (Dense)               (None, 1)                 506       
                                                                 
Total params: 1,067,066
Trainable params: 1,067,066
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=2122.902, rmse=46.075, r2=0.214; v_loss=3661.737, v_rmse=60.512, v_r2=0.016; 
E 2	: loss=1572.519, rmse=39.655, r2=0.418; v_loss=2669.1

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x0000017901CA9D00>, <keras.callbacks.LambdaCallback object at 0x0000017903001040>], epochs=2, model=<function create_model at 0x000001790303C9D0>, model__activation1='tanh', model__dropout1=0.10771222326909816, model__layer1=505, model__learning_rate=0.0032806529941975817, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000179030136A0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000179035D7D30>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.1, verbose=0),
                     scaler=MinMaxScaler(), seq_length=70)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.641,RMSE=-24.886
Finished: 2022-10-16 08:11:55.591639


## Non-Linear RUL

Score: 0.9430289602358563  
Test: 0.857
```
('basemodel__batch_size', 32),
('basemodel__epochs', 50),
('basemodel__model__activation', 'tanh'),
('basemodel__model__dropout', 0.1),
('basemodel__model__layer1', 512),
('basemodel__model__learning_rate', 0.0001),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('clip_y', 105),
('scaler', MinMaxScaler()),
('seq_length', 68)
```



In [ ]:
SEQ_LENGTH=68
CLIP=105

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=MinMaxScaler(),
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=50,
                           model__activation1='tanh',
                           model__dropout1=0.1, 
                           model__layer1=512, 
                           model__learning_rate=0.0001,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_3 (Masking)         (None, 68, 22)            0         
                                                                 
 lstm_3 (LSTM)               (None, 512)               1095680   
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 513       
                                                                 
Total params: 1,096,193
Trainable params: 1,096,193
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=2982.448, rmse=54.612, r2=-1.419; v_loss=2215.075, v_rmse=47.065, v_r2=-0.774; 
E 2	: loss=1623.784, rmse=40.296, r2=-0.317; v_loss=1706.530, v_rmse=41.310, v_r2=-0.

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000247B9A42760>, <keras.callbacks.LambdaCallback object at 0x00000247B9AC3D00>], epochs=50, model=<function create_model at 0x00000247B9B5CB80>, model__activation1='tanh', model__dropout1=0.1, model__layer1=512, model__learning_rate=0.0001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000247CA9147C0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000248EA003AC0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                     clip_y=105, scaler=MinMaxScaler(), seq_length=68)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.869,RMSE=-12.909
Finished: 2022-10-16 07:46:45.185856


# LSTM-Dense-1

## Linear RUL



Score: 0.8521617501513421  
Test: 0.793
```
('basemodel__batch_size', 239),
('basemodel__epochs', 45),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'relu'),
('basemodel__model__dropout1', 0.8360008606718795),
('basemodel__model__dropout2', 0.8326359552620336),
('basemodel__model__layer1', 493),
('basemodel__model__layer2', 469),
('basemodel__model__learning_rate', 0.0013568819509124186),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1967801723709445),
('scaler', StandardScaler()),
('seq_length', 74)
```



In [ ]:
SEQ_LENGTH=74
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=239,
                           epochs=45,
                           model__activation1='tanh',
                           model__activation2='relu',
                           model__dropout1=0.8360008606718795, 
                           model__dropout2=0.8326359552620336, 
                           model__layer1=493, 
                           model__layer2=469, 
                           model__learning_rate=0.0013568819509124186,
                           model__optim=Adam,
                           validation_split=0.1967801723709445, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 74, 22)            0         
                                                                 
 lstm (LSTM)                 (None, 493)               1017552   
                                                                 
 dropout (Dropout)           (None, 493)               0         
                                                                 
 dense (Dense)               (None, 469)               231686    
                                                                 
 dropout_1 (Dropout)         (None, 469)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 470       
                                                                 
Total params: 1,249,708
Trainable params: 1,249,708
Non-

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=239, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000197C771EB80>, <keras.callbacks.LambdaCallback object at 0x00000197C7753BE0>], epochs=45, model=<function create_model at 0x00000197C90B6EE0>, model__activation1='tanh', model__activation2='relu', model__dropout1=0.8360008606718795, model__dropout...r2=469, model__learning_rate=0.0013568819509124186, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000197C7AAFC40>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000197C92950D0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1967801723709445, verbose=0),
                     seq_length=74)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.793,RMSE=-18.928
Finished: 2022-10-16 08:19:44.587007


## Non-Linear RUL


Score: 0.917847894602053  
Test: 0.883
```
('basemodel__batch_size', 174),
('basemodel__epochs', 21),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__dropout1', 0.5157104351784535),
('basemodel__model__dropout2', 0.7126675967759073),
('basemodel__model__layer1', 125),
('basemodel__model__layer2', 386),
('basemodel__model__learning_rate', 0.0017804516876459917),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.23205313885059345),
('clip_y', 140),
('scaler', StandardScaler()),
('seq_length', 80)
```


In [ ]:
SEQ_LENGTH=80
CLIP=140

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=174,
                           epochs=21,
                           model__activation1='tanh',
                           model__activation2='selu',
                           model__dropout1=0.5157104351784535, 
                           model__dropout2=0.7126675967759073, 
                           model__layer1=125, 
                           model__layer2=386, 
                           model__learning_rate=0.0017804516876459917,
                           model__optim=Adam,
                           validation_split=0.23205313885059345, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [ ]:
model.fit(X_train_, y_train)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 80, 22)            0         
                                                                 
 lstm_1 (LSTM)               (None, 125)               74000     
                                                                 
 dropout_2 (Dropout)         (None, 125)               0         
                                                                 
 dense_2 (Dense)             (None, 386)               48636     
                                                                 
 dropout_3 (Dropout)         (None, 386)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 387       
                                                                 
Total params: 123,023
Trainable params: 123,023
Non-tr

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=174, callbacks=[<keras.callbacks.EarlyStopping object at 0x00000197C771EB80>, <keras.callbacks.LambdaCallback object at 0x00000197C7753BE0>], epochs=21, model=<function create_model at 0x00000197C90B6EE0>, model__activation1='tanh', model__activation2='selu', model__dropout1=0.5157104351784535, model__dropout...__learning_rate=0.0017804516876459917, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000197D6A3C430>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000019787C863A0>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.23205313885059345, verbose=0),
                     clip_y=140, seq_length=80)

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.883,RMSE=-14.201
Finished: 2022-10-16 08:20:41.062244


# LSTM-Dense-2

## Linear RUL



Score: 0.7968129295270682  
Test: 0.767
```
('basemodel__batch_size', 33),
('basemodel__epochs', 32),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'relu'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.22285374911306066),
('basemodel__model__dropout2', 0.7374780395679955),
('basemodel__model__dropout3', 0.3869949529630097),
('basemodel__model__layer1', 224),
('basemodel__model__layer2', 512),
('basemodel__model__layer3', 396),
('basemodel__model__learning_rate', 0.0006807885804241826),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.6313983549045777),
('scaler', MinMaxScaler()),
('seq_length', 95)
```




In [44]:
SEQ_LENGTH=95
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=MinMaxScaler(),
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=33,
                           epochs=32,
                           model__activation1='tanh',
                           model__activation2='relu',
                           model__activation3='selu',
                           model__dropout1=0.22285374911306066, 
                           model__dropout2=0.7374780395679955,
                           model__dropout3=0.3869949529630097, 
                           model__layer1=224, 
                           model__layer2=512, 
                           model__layer3=396, 
                           model__learning_rate=0.0006807885804241826,
                           model__optim=Adam,
                           validation_split=0.6313983549045777, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [45]:
model.fit(X_train_, y_train)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 95, 22)            0         
                                                                 
 lstm_2 (LSTM)               (None, 224)               221312    
                                                                 
 dropout_2 (Dropout)         (None, 224)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               115200    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 396)               203148    
                                                                 
 dropout_4 (Dropout)         (None, 396)              

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=33, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002120880ADC0>, <keras.callbacks.LambdaCallback object at 0x0000021208813CD0>], epochs=32, model=<function create_model at 0x000002123A5289D0>, model__activation1='tanh', model__activation2='relu', model__activation3='selu', model__dropout1=0.222853..._rate=0.0006807885804241826, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x0000021373F822E0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000021373FA2B50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.6313983549045777, verbose=0),
                     scaler=MinMaxScaler(), seq_length=95)

In [46]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.767,RMSE=-20.064
Finished: 2022-10-17 13:42:24.798953


## Non-Linear RUL


Score: 0.8303738924314298  
Test: 0.724
```
('basemodel__batch_size', 70),
('basemodel__epochs', 45),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'relu'),
('basemodel__model__activation3', 'tanh'),
('basemodel__model__dropout1', 0.677716159846799),
('basemodel__model__dropout2', 0.2905479330833355),
('basemodel__model__dropout3', 0.25279167191355223),
('basemodel__model__layer1', 177),
('basemodel__model__layer2', 470),
('basemodel__model__layer3', 221),
('basemodel__model__learning_rate', 0.009128147835708741),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.643380976472796),
('clip_y', 98),
('scaler', StandardScaler()),
('seq_length', 58)
```



In [47]:
SEQ_LENGTH=58
CLIP=98

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=70,
                           epochs=45,
                           model__activation1='tanh',
                           model__activation2='relu',
                           model__activation3='tanh',
                           model__dropout1=0.677716159846799, 
                           model__dropout2=0.2905479330833355,
                           model__dropout3=0.25279167191355223, 
                           model__layer1=177, 
                           model__layer2=470, 
                           model__layer3=221, 
                           model__learning_rate=0.009128147835708741,
                           model__optim=RMSprop,
                           validation_split=0.643380976472796, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [48]:
model.fit(X_train_, y_train)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_2 (Masking)         (None, 58, 22)            0         
                                                                 
 lstm_3 (LSTM)               (None, 177)               141600    
                                                                 
 dropout_5 (Dropout)         (None, 177)               0         
                                                                 
 dense_4 (Dense)             (None, 470)               83660     
                                                                 
 dropout_6 (Dropout)         (None, 470)               0         
                                                                 
 dense_5 (Dense)             (None, 221)               104091    
                                                                 
 dropout_7 (Dropout)         (None, 221)              

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=70, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002120880ADC0>, <keras.callbacks.LambdaCallback object at 0x0000021208813CD0>], epochs=45, model=<function create_model at 0x000002123A5289D0>, model__activation1='tanh', model__activation2='relu', model__activation3='tanh', model__dropout1=0.677716...learning_rate=0.009128147835708741, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000212181858E0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x0000021373F82880>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.643380976472796, verbose=0),
                     clip_y=98, seq_length=58)

In [51]:
model.score(X_train_, y_train)

0.8099047216294826

In [50]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.724,RMSE=-17.624
Finished: 2022-10-17 13:48:04.292748


# LSTM-LSTM-Dense

## Linear RUL



Score: 0.8491646332519348  
Test: 0.705
```
('basemodel__batch_size', 325),
             ('basemodel__epochs', 37),
             ('basemodel__model__activation1', 'tanh'),
             ('basemodel__model__activation2', 'tanh'),
             ('basemodel__model__activation3', 'sigmoid'),
             ('basemodel__model__dropout1', 0.5999695396702547),
             ('basemodel__model__dropout2', 0.2824705080924561),
             ('basemodel__model__dropout3', 0.5749415465530767),
             ('basemodel__model__layer1', 512),
             ('basemodel__model__layer2', 58),
             ('basemodel__model__layer3', 438),
             ('basemodel__model__learning_rate', 0.007448084633524131),
             ('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
             ('basemodel__validation_split', 0.10991019995076438),
             ('scaler', StandardScaler()),
             ('seq_length', 93)
```




In [89]:
SEQ_LENGTH=93
CLIP=-1

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=StandardScaler(),
        basemodel=
            KerasRegressor(model=create_model,
                           model__second_dense=False,
                           batch_size=325,
                           epochs=37,
                           model__activation1='tanh',
                           model__activation2='tanh',
                           model__activation3='sigmoid',
                           model__dropout1=0.5999695396702547, 
                           model__dropout2=0.2824705080924561,
                           model__dropout3=0.5749415465530767, 
                           model__layer1=512, 
                           model__layer2=58, 
                           model__layer3=438, 
                           model__learning_rate=0.007448084633524131,
                           model__optim=RMSprop,
                           validation_split=0.10991019995076438, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [90]:
model.fit(X_train_, y_train)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_8 (Masking)         (None, 93, 22)            0         
                                                                 
 lstm_14 (LSTM)              (None, 93, 512)           1095680   
                                                                 
 dropout_23 (Dropout)        (None, 93, 512)           0         
                                                                 
 lstm_15 (LSTM)              (None, 58)                132472    
                                                                 
 dropout_24 (Dropout)        (None, 58)                0         
                                                                 
 dense_17 (Dense)            (None, 438)               25842     
                                                                 
 dropout_25 (Dropout)        (None, 438)              

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=325, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002120880ADC0>, <keras.callbacks.LambdaCallback object at 0x0000021208813CD0>], epochs=37, model=<function create_model at 0x000002123A5289D0>, model__activation1='tanh', model__activation2='tanh', model__activation3='sigmoid', model__dropout1=0.59...7448084633524131, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x00000213739BBC10>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000213739BBAF0>], model__optim=<class 'keras.optimizer_v2.rmsprop.RMSprop'>, model__second_dense=False, print_summary=True, validation_split=0.10991019995076438, verbose=0),
                     seq_length=93)

In [92]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.708,RMSE=-22.445
Finished: 2022-10-17 14:12:13.939541


## Non-Linear RUL

Score: 0.9505741887137583  
Test: 0.
```
('basemodel__batch_size', 491),
             ('basemodel__epochs', 35),
             ('basemodel__model__activation1', 'tanh'),
             ('basemodel__model__activation2', 'tanh'),
             ('basemodel__model__activation3', 'selu'),
             ('basemodel__model__dropout1', 0.269131429819357),
             ('basemodel__model__dropout2', 0.1000488406021388),
             ('basemodel__model__dropout3', 0.3965483901301635),
             ('basemodel__model__layer1', 311),
             ('basemodel__model__layer2', 103),
             ('basemodel__model__layer3', 397),
             ('basemodel__model__learning_rate', 0.004499578015509351),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.24810164977060487),
             ('clip_y', 102),
             ('scaler', MinMaxScaler()),
             ('seq_length', 82)
```




In [93]:
SEQ_LENGTH=82
CLIP=102

model = LSTMWrapperRegressor(
        clip_y=CLIP, seq_length=SEQ_LENGTH, scaler=MinMaxScaler(),
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=491,
                           epochs=35,
                           model__activation1='tanh',
                           model__activation2='tanh',
                           model__activation3='selu',
                           model__dropout1=0.269131429819357, 
                           model__dropout2=0.1000488406021388,
                           model__dropout3=0.3965483901301635, 
                           model__layer1=311, 
                           model__layer2=103, 
                           model__layer3=397, 
                           model__learning_rate=0.004499578015509351,
                           model__optim=Adam,
                           validation_split=0.24810164977060487, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )

In [94]:
model.fit(X_train_, y_train)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_9 (Masking)         (None, 82, 22)            0         
                                                                 
 lstm_16 (LSTM)              (None, 311)               415496    
                                                                 
 dropout_26 (Dropout)        (None, 311)               0         
                                                                 
 dense_19 (Dense)            (None, 103)               32136     
                                                                 
 dropout_27 (Dropout)        (None, 103)               0         
                                                                 
 dense_20 (Dense)            (None, 397)               41288     
                                                                 
 dropout_28 (Dropout)        (None, 397)              

LSTMWrapperRegressor(basemodel=KerasRegressor(batch_size=491, callbacks=[<keras.callbacks.EarlyStopping object at 0x000002120880ADC0>, <keras.callbacks.LambdaCallback object at 0x0000021208813CD0>], epochs=35, model=<function create_model at 0x000002123A5289D0>, model__activation1='tanh', model__activation2='tanh', model__activation3='selu', model__dropout1=0.26913...499578015509351, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x0000021373BFC0D0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x00000213E382EE50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.24810164977060487, verbose=0),
                     clip_y=102, scaler=MinMaxScaler(), seq_length=82)

In [95]:
# Scaling and formatting test data
test_sc = scale_test(test,model)
test_wr = gen_test_wrapper(test_sc, SEQ_LENGTH, cols=model.seq_cols)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_wr))
print("Finished:", datetime.datetime.now())

R2=0.930,RMSE=-9.209
Finished: 2022-10-17 14:13:38.475811
